In [ ]:
pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 34.6 MB/s eta 0:00:00


In [ ]:
import matplotlib.pyplot as plt



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator




'''This code is for creating the model'''


image_x, image_y = 200, 200
batch_size = 64 #he number of samples processed during training
train_dir = "/content/drive/MyDrive/Chord Analyser/chords"


#creating CNN model architechture
def keras_model(image_x, image_y):
    num_of_classes = 5
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(image_x, image_y, 1), activation='relu')) #performing convolution on the input image
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')) #reducing the size of the feature maps obtained from previous layer
    model.add(Conv2D(64, (5, 5), activation='relu')) #adding another convolution layer to extract more complex features
    model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same')) #adding another pooling layer to reduce the size of feature maps
    model.add(Flatten()) #flattening the previous layer into a one-dimensional vector
    model.add(Dense(1024, activation='relu')) #a fully connected layer with 1024 units
    model.add(Dropout(0.6)) #randomly drops some of the neurons in previous layer to prevent overfitting
    model.add(Dense(num_of_classes, activation='softmax')) #output layer with softmax activation which genrates output probabilities

    #compiling the modal and saving the best model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    # filepath = "guitar_learner_final.h5"
    filepath = "guitar_learner_final.keras"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    return model, callbacks_list


def main():
    #setting up the dataset for creating training and testing sets
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        rotation_range=15,
        zoom_range=0.2,
        horizontal_flip=False,
        validation_split=0.2,
        fill_mode='nearest')


    #Training dataset 80%
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_x, image_y),
        color_mode="grayscale",
        batch_size=batch_size,
        seed=42,
        class_mode='categorical',
        subset="training")

    class_indices = train_generator.class_indices
    print(class_indices)

    #Testing dataset 20%
    validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_x, image_y),
        color_mode="grayscale",
        batch_size=batch_size,
        seed=42,
        class_mode='categorical',
        subset="validation")

    model, callbacks_list = keras_model(image_x, image_y)

    print(callbacks_list)
    #model.fit_generator(train_generator, epochs=5, validation_data=validation_generator)
    #early_stopping = callbacks.EarlyStopping(monitor="val_accuracy",mode = "max",patience=5,verbose=0,restore_best_weights=True)

    #starts training with 25 epochs
    history = model.fit_generator(train_generator, epochs=25, validation_data=validation_generator)
    #scores = model.evaluate_generator(generator=validation_generator, steps=64)

    #printing results
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    scores = model.evaluate_generator(generator=validation_generator, steps=64)

    print("CNN Error: %.2f%%" % (100 - scores[1] * 100))
    print("Accuracy: %.2f%%" % (scores[1] * 100))

    model_json = model.to_json()
    with open("model_final.json", "w") as json_file:
        json_file.write(model_json)

    model.save('guitar_learner_final.h5')




main()

Found 32008 images belonging to 4 classes.
{'C': 0, 'D': 1, 'Em': 2, 'G': 3}
Found 8002 images belonging to 4 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 196, 196, 32)        │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 98, 98, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 94, 94, 64)          │          51,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 19, 19, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 23104)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │      23,659,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │           5,125 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,716,741 (90.47 MB)

 Trainable params: 23,716,741 (90.47 MB)

 Non-trainable params: 0 (0.00 B)

AttributeError: 'Sequential' object has no attribute 'fit_generator'

In [ ]:
import matplotlib.pyplot as plt
# from keras.callbacks import ModelCheckpoint
# from keras.layers import Dense, Flatten, Conv2D
# from keras.layers import MaxPooling2D, Dropout
# from keras.models import Sequential
# from keras.preprocessing.image import ImageDataGenerator
# from tensorflow.preprocessing.image import ImageDataGenerator
# from keras import callbacks

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator




'''This code is for creating the model'''


image_x, image_y = 200, 200
batch_size = 64 #he number of samples processed during training
train_dir = "/content/drive/MyDrive/Chord Analyser/chords"


#creating CNN model architechture
def keras_model(image_x, image_y):
    num_of_classes = 5
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(image_x, image_y, 1), activation='relu')) #performing convolution on the input image
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')) #reducing the size of the feature maps obtained from previous layer
    model.add(Conv2D(64, (5, 5), activation='relu')) #adding another convolution layer to extract more complex features
    model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same')) #adding another pooling layer to reduce the size of feature maps
    model.add(Flatten()) #flattening the previous layer into a one-dimensional vector
    model.add(Dense(1024, activation='relu')) #a fully connected layer with 1024 units
    model.add(Dropout(0.6)) #randomly drops some of the neurons in previous layer to prevent overfitting
    model.add(Dense(num_of_classes, activation='softmax')) #output layer with softmax activation which genrates output probabilities

    #compiling the modal and saving the best model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    # filepath = "guitar_learner_final.h5"
    filepath = "guitar_learner_final.keras"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    return model, callbacks_list


def main():
    #setting up the dataset for creating training and testing sets
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        rotation_range=15,
        zoom_range=0.2,
        horizontal_flip=False,
        validation_split=0.2,
        fill_mode='nearest')


    #Training dataset 80%
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_x, image_y),
        color_mode="grayscale",
        batch_size=batch_size,
        seed=42,
        class_mode='categorical',
        subset="training")

    class_indices = train_generator.class_indices
    print(class_indices)

    #Testing dataset 20%
    validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_x, image_y),
        color_mode="grayscale",
        batch_size=batch_size,
        seed=42,
        class_mode='categorical',
        subset="validation")

    model, callbacks_list = keras_model(image_x, image_y)

    print(callbacks_list)
    #model.fit_generator(train_generator, epochs=5, validation_data=validation_generator)
    #early_stopping = callbacks.EarlyStopping(monitor="val_accuracy",mode = "max",patience=5,verbose=0,restore_best_weights=True)

    #starts training with 25 epochs
    # Replacing 'fit_generator' with 'fit'
    history = model.fit(train_generator, epochs=25, validation_data=validation_generator)
    #scores = model.evaluate_generator(generator=validation_generator, steps=64)

    #printing results
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    # Replacing 'evaluate_generator' with 'evaluate'
    scores = model.evaluate(validation_generator, steps=64)

    print("CNN Error: %.2f%%" % (100 - scores[1] * 100))
    print("Accuracy: %.2f%%" % (scores[1] * 100))

    model_json = model.to_json()
    with open("model_final.json", "w") as json_file:
        json_file.write(model_json)

    model.save('guitar_learner_final.h5')




main()

Found 2855 images belonging to 4 classes.
{'C': 0, 'D': 1, 'Em': 2, 'G': 3}
Found 713 images belonging to 4 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 196, 196, 32)        │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 98, 98, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 94, 94, 64)          │          51,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 19, 19, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 23104)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1024)                │      23,659,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 5)                   │           5,125 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,716,741 (90.47 MB)

 Trainable params: 23,716,741 (90.47 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 4), output.shape=(None, 5)

In [ ]:
import matplotlib.pyplot as plt
# from keras.callbacks import ModelCheckpoint
# from keras.layers import Dense, Flatten, Conv2D
# from keras.layers import MaxPooling2D, Dropout
# from keras.models import Sequential
# from keras.preprocessing.image import ImageDataGenerator
# from tensorflow.preprocessing.image import ImageDataGenerator
# from keras import callbacks

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import model_from_json



'''This code is for creating the model'''


image_x, image_y = 200, 200
# batch_size = 64 #he number of samples processed during training
batch_size = 32
train_dir = "/content/drive/MyDrive/Chord Analyser/chords"
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(image_x, image_y, 3))


#creating CNN model architechture
def keras_model(image_x, image_y, num_of_classes): # Added num_of_classes as an argument
    # num_of_classes = 5  # Removed hardcoded value
    # model = Sequential()
    # model.add(Conv2D(32, (5, 5), input_shape=(image_x, image_y, 1), activation='relu')) #performing convolution on the input image
    # model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')) #reducing the size of the feature maps obtained from previous layer
    # model.add(Conv2D(64, (5, 5), activation='relu')) #adding another convolution layer to extract more complex features
    # model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same')) #adding another pooling layer to reduce the size of feature maps
    # model.add(Flatten()) #flattening the previous layer into a one-dimensional vector
    # model.add(Dense(1024, activation='relu')) #a fully connected layer with 1024 units
    # model.add(Dropout(0.6)) #randomly drops some of the neurons in previous layer to prevent overfitting
    # model.add(Dense(num_of_classes, activation='softmax')) #output layer with softmax activation which genrates output probabilities

    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(image_x, image_y, 3))
    base_model.trainable = False  # Optional: Freeze the base model layers initially

    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(num_of_classes, activation='softmax'))

    #compiling the modal and saving the best model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    # filepath = "guitar_learner_final.h5"
    filepath = "guitar_learner_final.keras"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    return model, callbacks_list


def main():
    #setting up the dataset for creating training and testing sets
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        rotation_range=15,
        zoom_range=0.2,
        horizontal_flip=False,
        validation_split=0.2,
        fill_mode='nearest')


    #Training dataset 80%
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_x, image_y),
        # color_mode="grayscale",
        color_mode="rgb",
        batch_size=batch_size,
        seed=42,
        class_mode='categorical',
        subset="training")

    class_indices = train_generator.class_indices
    print(class_indices)

    num_classes = len(class_indices) # Get the actual number of classes from your data

    #Testing dataset 20%
    validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_x, image_y),
        # color_mode="grayscale",
        color_mode="rgb",
        batch_size=batch_size,
        seed=42,
        class_mode='categorical',
        subset="validation")

    model, callbacks_list = keras_model(image_x, image_y, num_classes) # Pass num_classes to keras_model

    print(callbacks_list)
    #model.fit_generator(train_generator, epochs=5, validation_data=validation_generator)
    #early_stopping = callbacks.EarlyStopping(monitor="val_accuracy",mode = "max",patience=5,verbose=0,restore_best_weights=True)

    #starts training with 25 epochs
    # Replacing 'fit_generator' with 'fit'
    print('Training Begins')
    history = model.fit(train_generator, epochs=5, validation_data=validation_generator)
    #scores = model.evaluate_generator(generator=validation_generator, steps=64)

    #printing results
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    # Replacing 'evaluate_generator' with 'evaluate'
    scores = model.evaluate(validation_generator, steps=64)

    print("CNN Error: %.2f%%" % (100 - scores[1] * 100))
    print("Accuracy: %.2f%%" % (scores[1] * 100))

    model_json = model.to_json()
    with open("model_final.json", "w") as json_file:
        json_file.write(model_json)

    model.save('guitar_learner_final.h5')




main()

<ipython-input-2-ba28d0dd66cc>:26: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(image_x, image_y, 3))


Found 32008 images belonging to 4 classes.
{'C': 0, 'D': 1, 'Em': 2, 'G': 3}
Found 8002 images belonging to 4 classes.


<ipython-input-2-ba28d0dd66cc>:42: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(image_x, image_y, 3))


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 62720)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1024)                │      64,226,304 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 4)                   │           4,100 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

...

 Non-trainable params: 2,257,984 (8.61 MB)

Training Begins
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  20/1001 ━━━━━━━━━━━━━━━━━━━━ 4:34:52 17s/step - accuracy: 0.3721 - loss: 58.0942

KeyboardInterrupt: 

In [ ]:
# !pip install mediapipe

import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import model_from_json
from tensorflow.keras import mixed_precision
from tensorflow.keras.applications import EfficientNetB0
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)



'''This code is for creating the model'''


image_x, image_y = 200, 200
batch_size = 16
train_dir = "/content/drive/MyDrive/Chord Analyser/chordsss"


def keras_model(image_x, image_y, num_of_classes):
    # base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(image_x, image_y, 3))
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_x, image_y, 3))
    base_model.trainable = False

    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(num_of_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    # filepath = "guitar_learner_final.keras"
    # checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    filepath = "guitar_learner_{epoch:02d}.keras"  # Save with epoch number
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    return model, callbacks_list


def main():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        rotation_range=15,
        zoom_range=0.2,
        horizontal_flip=False,
        validation_split=0.2,
        fill_mode='nearest')


    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_x, image_y),
        color_mode="rgb",
        batch_size=batch_size,
        seed=42,
        class_mode='categorical',
        subset="training")

    class_indices = train_generator.class_indices
    print(class_indices)

    num_classes = len(class_indices)

    validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_x, image_y),
        color_mode="rgb",
        batch_size=batch_size,
        seed=42,
        class_mode='categorical',
        subset="validation")

    # Load model if resuming training
    try:
        with open('model_architecture.json', 'r') as json_file:
            loaded_model_json = json_file.read()
        model = model_from_json(loaded_model_json)

        model.load_weights('guitar_learner_final.keras')

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        print("Loaded model from disk")

    except FileNotFoundError:
        print("Starting training from scratch")
        model, callbacks_list = keras_model(image_x, image_y, num_classes)

    if model is None:
      model, callbacks_list = keras_model(image_x, image_y, num_classes)
      history = model.fit(train_generator, epochs=5, validation_data=validation_generator)
    else:
       # Continue training with loaded model
       history = model.fit(train_generator, initial_epoch=5, epochs=10, validation_data=validation_generator)

    # Save model architecture
    # model_json = model.to_json()
    # with open("model_architecture.json", "w") as json_file:
    #     json_file.write(model_json)

    model_json = model.to_json()
    with open("/content/drive/MyDrive/Chord Analyser/model_architecture.json", "w") as json_file:  # Add the desired location
        json_file.write(model_json)

    # Printing and saving results
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    scores = model.evaluate(validation_generator, steps=64)

    print("CNN Error: %.2f%%" % (100 - scores[1] * 100))
    print("Accuracy: %.2f%%" % (scores[1] * 100))

    model.save('/content/drive/MyDrive/Chord Analyser/guitar_learner_final.h5')  # Save the entire model


main()

Found 16008 images belonging to 4 classes.
{'C': 0, 'D': 1, 'Em': 2, 'G': 3}
Found 4002 images belonging to 4 classes.
Starting training from scratch
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)          │ (None, 7, 7, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 62720)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │      64,226,304 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │           4,100 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 68,279,975 (260.47 MB)

 Trainable params: 64,230,404 (245.02 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

Epoch 6/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 473/1001 ━━━━━━━━━━━━━━━━━━━━ 1:02:24 7s/step - accuracy: 0.2468 - loss: 11.8370

Below is training for 20000 images only

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, GaussianNoise
from tensorflow.keras.layers import MaxPooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

# Image dimensions and batch size
image_x, image_y = 200, 200
batch_size = 32
train_dir = "/content/drive/MyDrive/Chord Analyser/chords"

# Model creation function with smaller layers, noise, and more dropout
def keras_model(image_x, image_y, num_of_classes):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(image_x, image_y, 3))
    base_model.trainable = False

    model = Sequential()
    model.add(base_model)
    model.add(GaussianNoise(0.2))  # Increased noise
    model.add(Flatten())
    model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  # Smaller Dense layer with L2 regularization
    model.add(Dropout(0.7))  # Higher dropout
    model.add(Dense(num_of_classes, activation='softmax'))

    # Use a smaller learning rate for optimization
    optimizer = Adam(learning_rate=0.00005)  # Smaller learning rate
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    model.summary()

    return model

def main():
    # Data augmentation setup with more aggressive transformations
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        width_shift_range=0.4,  # Increased width shift
        height_shift_range=0.4,  # Increased height shift
        shear_range=0.4,  # Increased shear range
        rotation_range=25,  # Increased rotation range
        zoom_range=0.3,  # Increased zoom range
        horizontal_flip=True,  # Flip images horizontally
        validation_split=0.2,
        fill_mode='nearest')

    # Training data generator
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_x, image_y),
        color_mode="rgb",
        batch_size=batch_size,
        seed=42,
        class_mode='categorical',
        subset="training",
        shuffle=True  # Keep shuffle True to add randomness
    )

    class_indices = train_generator.class_indices
    print(class_indices)

    num_classes = len(class_indices)

    # Validation data generator
    validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_x, image_y),
        color_mode="rgb",
        batch_size=batch_size,
        seed=42,
        class_mode='categorical',
        subset="validation",
        shuffle=False
    )

    # Prepare data in numpy arrays to handle large datasets
    X_train, y_train = next(train_generator)  # Get the first batch of data
    for i in range(int(1000 / batch_size) - 1):  # Collect a smaller dataset (e.g., 2000 images)
        X_temp, y_temp = next(train_generator)
        X_train = np.concatenate((X_train, X_temp))
        y_train = np.concatenate((y_train, y_temp))

    X_val, y_val = next(validation_generator)  # Get the first batch of data
    for i in range(int(250 / batch_size) - 1):  # Collect smaller dataset for validation
        X_temp, y_temp = next(validation_generator)
        X_val = np.concatenate((X_val, X_temp))
        y_val = np.concatenate((y_val, y_temp))

    # Load model if resuming training
    try:
        with open('model_architecture.json', 'r') as json_file:
            loaded_model_json = json_file.read()
        model = model_from_json(loaded_model_json)
        model.load_weights('guitar_learner_final.keras')
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        print("Loaded model from disk")

    except FileNotFoundError:
        print("Starting training from scratch")
        model = keras_model(image_x, image_y, num_classes)

    # Train the model (limit epochs)
    history = model.fit(X_train, y_train, epochs=25, validation_data=(X_val, y_val))  # Limit to fewer epochs

    # Save model architecture
    model_json = model.to_json()
    with open("/content/drive/MyDrive/Chord Analyser/model_architecture.json", "w") as json_file:
        json_file.write(model_json)

    # Plotting results
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    # Final evaluation on the validation set
    scores = model.evaluate(X_val, y_val, steps=64)
    print("CNN Error: %.2f%%" % (100 - scores[1] * 100))
    print("Accuracy: %.2f%%" % (scores[1] * 100))

    model.save('/content/drive/MyDrive/Chord Analyser/guitar_learner_final.h5')

main()


Found 32008 images belonging to 4 classes.
{'C': 0, 'D': 1, 'Em': 2, 'G': 3}
Found 8002 images belonging to 4 classes.
Starting training from scratch


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)          │ (None, 7, 7, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gaussian_noise_3 (GaussianNoise)     │ (None, 7, 7, 1280)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 62720)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 512)                 │      32,113,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 4)                   │           2,052 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 36,164,775 (137.96 MB)

 Trainable params: 32,115,204 (122.51 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

Epoch 1/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 132s 4s/step - accuracy: 0.2413 - loss: 11.9043 - val_accuracy: 1.0000 - val_loss: 10.0892
Epoch 2/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 135s 4s/step - accuracy: 0.2836 - loss: 10.1538 - val_accuracy: 0.0000e+00 - val_loss: 9.4528
Epoch 3/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 119s 4s/step - accuracy: 0.2405 - loss: 9.0592 - val_accuracy: 0.0000e+00 - val_loss: 8.4723
Epoch 4/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 138s 4s/step - accuracy: 0.2601 - loss: 8.2192 - val_accuracy: 0.0000e+00 - val_loss: 7.7591
Epoch 5/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 148s 4s/step - accuracy: 0.2793 - loss: 7.5216 - val_accuracy: 0.0000e+00 - val_loss: 7.1063
Epoch 6/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 141s 4s/step - accuracy: 0.2574 - loss: 6.9213 - val_accuracy: 0.0000e+00 - val_loss: 6.5281
Epoch 7/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 141s 4s/step - accuracy: 0.3016 - loss: 6.4061 - val_accuracy: 0.0000e+00 - val_loss: 6.0733
Epoch 8/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 143s 4s/step - accuracy: 0.2842 - loss: 5.9615

Testing Phase